# Solution for Assignment 5: Multi-period Lot-sizing Problem

This notebook solves the problems from Assignment 5, focusing on multi-period lot-sizing problems using different heuristic and optimal methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Get the absolute path of the current file and add parent directory to path
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

try:
    import gurobipy as gp
    from gurobipy import GRB, quicksum
    GUROBI_AVAILABLE = True
except ImportError:
    print("Gurobi not available. Some functionality will be limited.")
    GUROBI_AVAILABLE = False

## Problem Data

Let's define the problem data for the multi-period lot-sizing problem.

In [ ]:
# Problem data
demands = np.array([550, 200, 400, 110, 430, 980, 400, 300, 200, 650])
setup_cost = 10000
holding_cost = 0.2 * 120  # euro/unit/period

num_periods = len(demands)

print(f"Demands: {demands}")
print(f"Setup cost: {setup_cost}")
print(f"Holding cost: {holding_cost}")
print(f"Number of periods: {num_periods}")

## Helper Functions

Let's define helper functions for calculating costs and inventory levels.

In [ ]:
def calculate_total_cost(setup_decision, lot_size, demands, setup_cost, holding_cost):
    """Calculate the total cost of a given lot-sizing solution"""
    num_periods = len(demands)
    total_setup_cost = setup_cost * np.sum(setup_decision)
    
    # Calculate inventory levels
    inventory = np.zeros(num_periods)
    inventory[0] = lot_size[0] - demands[0]
    
    for t in range(1, num_periods):
        inventory[t] = inventory[t-1] + lot_size[t] - demands[t]
    
    total_holding_cost = holding_cost * np.sum(inventory)
    total_cost = total_setup_cost + total_holding_cost
    
    return total_cost

def calculate_inventory_levels(lot_size, demands):
    """Calculate inventory levels for a given lot-sizing solution"""
    num_periods = len(demands)
    inventory = np.zeros(num_periods)
    
    inventory[0] = lot_size[0] - demands[0]
    for t in range(1, num_periods):
        inventory[t] = inventory[t-1] + lot_size[t] - demands[t]
    
    return inventory

## Exercise 5(a): Least Unit Cost Heuristic

In [ ]:
def calculate_luc_criterion(t, z, demands, setup_cost, holding_cost):
    """Calculate the Least Unit Cost criterion for periods t through z"""
    holding_periods = [i for i in range(z - t + 1)]
    total_demand = np.sum(demands[t:z+1])
    
    if total_demand == 0:
        return float('inf')  # Avoid division by zero
    
    unit_cost = (setup_cost + holding_cost * np.sum(demands[t:z+1] * holding_periods)) / total_demand
    return unit_cost

def least_unit_cost_heuristic(demands, setup_cost, holding_cost):
    """Implement the Least Unit Cost heuristic for lot-sizing"""
    num_periods = len(demands)
    flag_setup = np.full(num_periods, False, dtype=bool)
    lot_size = np.zeros(num_periods)
    
    t = 0
    while t < num_periods:
        z = t
        c_opt = calculate_luc_criterion(t, z, demands, setup_cost, holding_cost)
        
        while z < num_periods - 1 and c_opt > calculate_luc_criterion(t, z + 1, demands, setup_cost, holding_cost):
            z += 1
            c_opt = calculate_luc_criterion(t, z, demands, setup_cost, holding_cost)
        
        flag_setup[t] = True
        lot_size[t] = np.sum(demands[t:z+1])
        t = z + 1
    
    return flag_setup, lot_size

# Apply Least Unit Cost heuristic
luc_setup, luc_lot_size = least_unit_cost_heuristic(demands, setup_cost, holding_cost)
luc_cost = calculate_total_cost(luc_setup, luc_lot_size, demands, setup_cost, holding_cost)

print("Least Unit Cost Heuristic")
print(f"Setup decision: {luc_setup}")
print(f"Lot-sizing decision: {luc_lot_size}")
print(f"Total costs: {luc_cost}")

## Exercise 5(b): Silver-Meal Heuristic

In [ ]:
def calculate_sm_criterion(t, z, demands, setup_cost, holding_cost):
    """Calculate the Silver-Meal criterion for periods t through z"""
    holding_periods = [i for i in range(z - t + 1)]
    
    period_cost = (setup_cost + holding_cost * np.sum(demands[t:z+1] * holding_periods)) / (z - t + 1)
    return period_cost

def silver_meal_heuristic(demands, setup_cost, holding_cost):
    """Implement the Silver-Meal heuristic for lot-sizing"""
    num_periods = len(demands)
    flag_setup = np.full(num_periods, False, dtype=bool)
    lot_size = np.zeros(num_periods)
    
    t = 0
    while t < num_periods:
        z = t
        c_opt = calculate_sm_criterion(t, z, demands, setup_cost, holding_cost)
        
        while z < num_periods - 1 and c_opt > calculate_sm_criterion(t, z + 1, demands, setup_cost, holding_cost):
            z += 1
            c_opt = calculate_sm_criterion(t, z, demands, setup_cost, holding_cost)
        
        flag_setup[t] = True
        lot_size[t] = np.sum(demands[t:z+1])
        t = z + 1
    
    return flag_setup, lot_size

# Apply Silver-Meal heuristic
sm_setup, sm_lot_size = silver_meal_heuristic(demands, setup_cost, holding_cost)
sm_cost = calculate_total_cost(sm_setup, sm_lot_size, demands, setup_cost, holding_cost)

print("Silver-Meal Heuristic")
print(f"Setup decision: {sm_setup}")
print(f"Lot-sizing decision: {sm_lot_size}")
print(f"Total costs: {sm_cost}")

## Exercise 5(c): Wagner-Whitin Algorithm

For computational simplicity, we'll use a subset of the periods for the Wagner-Whitin algorithm.

In [ ]:
def calculate_lot_sizes(setup_decision, demands):
    """Calculate lot sizes given setup decisions and demands"""
    num_periods = len(demands)
    lot_size = np.zeros(num_periods)
    
    last_setup = 0
    for t in range(1, num_periods + 1):
        if t == num_periods or setup_decision[t]:
            lot_size[last_setup] = np.sum(demands[last_setup:t])
            last_setup = t
    
    return lot_size

def wagner_whitin_algorithm(demands, setup_cost, holding_cost):
    """Implement the Wagner-Whitin dynamic programming algorithm"""
    num_periods = len(demands)
    
    # 2-d array consists of the total costs for all possible lot-sizing decisions
    costs = np.full((num_periods, num_periods), np.inf)
    
    # Base case: Order in the first period
    costs[0, 0] = setup_cost
    for t in range(1, num_periods):
        costs[0, t] = costs[0, t - 1] + t * holding_cost * demands[t]
    
    # Fill in the rest of the dynamic programming table
    for i in range(1, num_periods):
        # Cost of setting up production in period i
        costs[i, i] = np.min(costs[:, i-1]) + setup_cost
        
        # Cost of producing demand for future periods
        for t in range(i+1, num_periods):
            costs[i, t] = costs[i, t-1] + (t - i) * holding_cost * demands[t]
    
    # Print the costs matrix
    print("Costs for all possible lot-sizing decisions: ")
    print(costs)
    
    # Extract the optimal setup decisions
    index_opt = np.argmin(costs, axis=0)
    setup_decision = np.full(num_periods, False, dtype=bool)
    setup_decision[0] = True
    
    for t in range(1, num_periods):
        if index_opt[t] == index_opt[t-1]:
            setup_decision[t] = False
        else:
            setup_decision[t] = True
    
    # Calculate lot sizes based on setup decisions
    lot_size = calculate_lot_sizes(setup_decision, demands)
    
    # Calculate total cost
    total_cost = np.min(costs[:, -1])
    
    return setup_decision, lot_size, total_cost

# Use a smaller subset of periods for Wagner-Whitin demonstration
ww_demands = demands[:6]
ww_num_periods = len(ww_demands)
print(f"Using the first {ww_num_periods} periods for Wagner-Whitin: {ww_demands}\n")

ww_setup_decision, ww_lot_size, ww_total_cost = wagner_whitin_algorithm(ww_demands, setup_cost, holding_cost)

print("\nWagner-Whitin Algorithm")
print(f"Setup decision: {ww_setup_decision}")
print(f"Lot-sizing decision: {ww_lot_size}")
print(f"Total costs: {ww_total_cost}")

## Exercise 5(c) MILP: Mixed-Integer Linear Programming

In [ ]:
def milp_lot_sizing(demands, setup_cost, holding_cost):
    """Implement the lot-sizing problem using Mixed-Integer Linear Programming"""
    if not GUROBI_AVAILABLE:
        print("Gurobi not available. Cannot solve using MILP.")
        return None, None, None
    
    num_periods = len(demands)
    big_M = np.sum(demands)
    
    # Create model
    model = gp.Model("Wagner-Whitin")
    
    # Create variables
    lotsize = model.addVars(num_periods, vtype=GRB.INTEGER, name="lotsize")
    setup = model.addVars(num_periods, vtype=GRB.BINARY, name="setup indicator")
    inventories = model.addVars(num_periods, name="inventories")
    
    # Set objective
    model.setObjective(
        quicksum(
            setup[period] * setup_cost + holding_cost * inventories[period]
            for period in range(num_periods)
        ),
        GRB.MINIMIZE
    )
    
    # Inventory balance constraints
    model.addConstr(inventories[0] == lotsize[0] - demands[0])
    model.addConstrs(
        inventories[period] == lotsize[period] - demands[period] + inventories[period - 1]
        for period in range(1, num_periods)
    )
    
    # Logic constraints
    model.addConstrs(lotsize[period] <= big_M * setup[period] for period in range(num_periods))
    
    # Solve model
    model.optimize()
    
    # Extract results
    setup_decision = [bool(setup[i].X) for i in range(num_periods)]
    lot_size = [lotsize[i].X for i in range(num_periods)]
    total_cost = model.objVal
    
    return setup_decision, lot_size, total_cost

if GUROBI_AVAILABLE:
    milp_setup_decision, milp_lot_size, milp_total_cost = milp_lot_sizing(ww_demands, setup_cost, holding_cost)
    
    print("\nMILP Solution")
    print(f"Setup decision: {milp_setup_decision}")
    print(f"Lot-sizing decision: {milp_lot_size}")
    print(f"Total costs: {milp_total_cost}")

## Visualization and Comparison

In [ ]:
# Create plot for lot sizes
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.bar(range(len(demands)), demands, alpha=0.3, color='gray', label='Demands')
plt.bar(range(len(demands)), luc_lot_size, alpha=0.6, color='blue', label='LUC Lot Size')
plt.bar(range(len(demands)), sm_lot_size, alpha=0.6, color='green', label='SM Lot Size')

# For Wagner-Whitin, create a full-size array (first 6 periods only, rest zeros)
ww_lot_size_full = np.zeros(len(demands))
ww_lot_size_full[:len(ww_lot_size)] = ww_lot_size
plt.bar(range(len(demands)), ww_lot_size_full, alpha=0.6, color='red', label='WW Lot Size (first 6 periods)')

plt.xlabel('Period')
plt.ylabel('Units')
plt.title('Comparison of Lot-Sizing Methods')
plt.legend()
plt.grid(True, alpha=0.3)

# Create plot for inventory levels
plt.subplot(2, 1, 2)

# Calculate inventory levels for each method
luc_inventory = calculate_inventory_levels(luc_lot_size, demands)
sm_inventory = calculate_inventory_levels(sm_lot_size, demands)
ww_inventory = calculate_inventory_levels(ww_lot_size_full, demands)

plt.plot(range(len(demands)), luc_inventory, 'bo-', label=f'LUC (Cost: {luc_cost})')
plt.plot(range(len(demands)), sm_inventory, 'go-', label=f'SM (Cost: {sm_cost})')
plt.plot(range(len(demands)), ww_inventory, 'ro-', label=f'WW (First 6 periods)')

# Mark setup periods
for t in range(len(demands)):
    if luc_setup[t]:
        plt.axvline(x=t, color='blue', linestyle='--', alpha=0.5)
    if sm_setup[t]:
        plt.axvline(x=t, color='green', linestyle=':', alpha=0.5)
    if t < len(ww_setup_decision) and ww_setup_decision[t]:
        plt.axvline(x=t, color='red', linestyle='-.', alpha=0.5)

plt.xlabel('Period')
plt.ylabel('Inventory Level')
plt.title('Inventory Levels by Method')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Detailed Analysis of Results

Let's analyze the results from different lot-sizing methods.

In [ ]:
# Create a summary table of results
print("Summary of Results for the First 6 Periods:\n")
print("Period | Demand | LUC Lot Size | SM Lot Size | WW Lot Size")
print("-" * 60)

for t in range(len(ww_demands)):
    print(f"{t:6d} | {ww_demands[t]:6d} | {luc_lot_size[t]:11.0f} | {sm_lot_size[t]:10.0f} | {ww_lot_size[t]:10.0f}")

print("\nSetup Decisions:")
print(f"LUC: {luc_setup[:len(ww_demands)]}")
print(f"SM:  {sm_setup[:len(ww_demands)]}")
print(f"WW:  {ww_setup_decision}")

if GUROBI_AVAILABLE:
    print(f"MILP: {milp_setup_decision}")

print("\nTotal Costs for First 6 Periods:")
luc_cost_subset = calculate_total_cost(luc_setup[:len(ww_demands)], luc_lot_size[:len(ww_demands)], ww_demands, setup_cost, holding_cost)
sm_cost_subset = calculate_total_cost(sm_setup[:len(ww_demands)], sm_lot_size[:len(ww_demands)], ww_demands, setup_cost, holding_cost)

print(f"LUC: {luc_cost_subset}")
print(f"SM:  {sm_cost_subset}")
print(f"WW:  {ww_total_cost}")
if GUROBI_AVAILABLE:
    print(f"MILP: {milp_total_cost}")

## Comparing Solution Quality

Let's analyze the solution quality of the different approaches.

In [ ]:
# Compare solution quality 
if GUROBI_AVAILABLE and ww_total_cost == milp_total_cost:
    print("The Wagner-Whitin algorithm produces an optimal solution, identical to MILP.")

# Calculate optimality gaps
if GUROBI_AVAILABLE:
    optimal_cost = milp_total_cost  # Assuming MILP gives the optimal solution
    
    luc_gap = (luc_cost_subset - optimal_cost) / optimal_cost * 100
    sm_gap = (sm_cost_subset - optimal_cost) / optimal_cost * 100
    
    print(f"\nOptimality Gaps:")
    print(f"LUC: {luc_gap:.2f}%")
    print(f"SM:  {sm_gap:.2f}%")
    
    # Create a bar chart of the optimality gaps
    plt.figure(figsize=(8, 5))
    plt.bar(['LUC', 'SM', 'WW/MILP'], [luc_gap, sm_gap, 0])
    plt.ylabel('Optimality Gap (%)')
    plt.title('Optimality Gap of Lot-Sizing Methods')
    plt.grid(axis='y', alpha=0.3)
    plt.show()